In [ ]:
from google.colab import drive
import os

drive.mount('carga',force_remount=True)

Mounted at carga


In [ ]:
#CUSTOM_MODEL_NAME = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'
#CUSTOM_MODEL_NAME = 'facessd_mobilenet_v2_quantized_320x320_open_image_v4'   
CUSTOM_MODEL_NAME = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'

IMAGE_PATH = '/content/carga/MyDrive/workspace/images'
ANNOTATION_PATH = '/content/carga/MyDrive/workspace/annotations' 
OUTPUT_PATH = '/content/carga/MyDrive/workspace/tf1/'+ CUSTOM_MODEL_NAME + '/output'
EXPORT_PATH = '/content/carga/MyDrive/workspace/tf1/'+ CUSTOM_MODEL_NAME + '/export'

In [ ]:
# Install tflite_runtime package to evaluate the model.
!pip3 install https://github.com/google-coral/pycoral/releases/download/release-frogfish/tflite_runtime-2.5.0-cp37-cp37m-linux_x86_64.whl  

     |████████████████████████████████| 1.2MB 12.4MB/s 


In [ ]:
# Now we do evaluation on the tflite model.
import os
import numpy as np
from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate
from PIL import Image
from PIL import ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

# Creates tflite interpreter
interpreter = Interpreter('/content/model300.tflite')
# This exact code can be used to run inference on the edgetpu by simply creating 
# the instantialize the interpreter with libedgetpu delegates:
# interpreter = Interpreter(args.model, experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
print(interpreter.get_input_details())

interpreter.allocate_tensors()
interpreter.invoke() # warmup
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
width = input_details[0]['shape'][2]
height = input_details[0]['shape'][1]

def run_inference(interpreter, image):
  interpreter.set_tensor(input_details[0]['index'], image)
  interpreter.invoke()
  boxes = interpreter.get_tensor(output_details[0]['index'])[0]
  classes = interpreter.get_tensor(output_details[1]['index'])[0]
  scores = interpreter.get_tensor(output_details[2]['index'])[0]
  # num_detections = interpreter.get_tensor(output_details[3]['index'])[0]
  return boxes, classes, scores

test_image_paths = [os.path.join('/content/carga/MyDrive/workspace/test/', 'image{}.png'.format(i)) for i in range(0, 25)]
for image_path in test_image_paths:
  print('Evaluating:', image_path)
  image = Image.open(image_path)
  image_width, image_height = image.size
  draw = ImageDraw.Draw(image)
  resized_image = image.resize((width, height))
  np_image = np.asarray(resized_image)
  input_tensor = np.expand_dims(np_image, axis=0)
  # Run inference
  boxes, classes, scores = run_inference(interpreter, input_tensor)
  # Draw results on image
  colors = {0:(128, 255, 102), 1:(102, 255, 255)}
  labels = {0:'Mask', 1:'No_Mask'}
  for i in range(len(boxes)):
    if scores[i] > .7:
      ymin = int(max(1, (boxes[i][0] * image_height)))
      xmin = int(max(1, (boxes[i][1] * image_width)))
      ymax = int(min(image_height, (boxes[i][2] * image_height)))
      xmax = int(min(image_width, (boxes[i][3] * image_width)))
      draw.rectangle((xmin, ymin, xmax, ymax), width=7, outline=colors[int(classes[i])])
      draw.rectangle((xmin, ymin, xmax, ymin-10), fill=colors[int(classes[i])])
      text = labels[int(classes[i])] + ' ' + str(scores[i]*100) + '%'
      draw.text((xmin+2, ymin-10), text, fill=(0,0,0), width=2)
  display(image)
  